# Evolutionary Computation Assignment 3

- Krzysztof Szala 144571
- Vadym Repetskyi 155610


In [1]:
from utils import TspInstance, random_solution, weighted_regret
import numpy as np
import pandas as pd
import itertools

In [30]:
def cost_change_inter(tsp, solution, i, j):
    i = i[0][0]

    if i == 0:
        i_prev = len(solution) - 1
    else:
        i_prev = i - 1

    if i == len(solution) - 1:
        i_next = 0
    else:
        i_next = i + 1

    cost_change = (
        tsp.node_costs[j]
        - tsp.node_costs[i]
        - tsp.distance_matrix[solution[i_prev], solution[i]]
        - tsp.distance_matrix[solution[i], solution[i_next]]
        + tsp.distance_matrix[solution[i_prev], j]
        + tsp.distance_matrix[j, solution[i_next]]
    )
    return cost_change


def cost_change_intra_nodes(tsp, solution, i, j):
    # i is the index of the node to be replaced
    # j is the node to replace i
    # operations adding 4 new edge and removing 4 old edges
    # is it possible to know, which we don't have to remove?

    i = i[0][0]
    j = j[0][0]

    if i == 0:
        i_prev = -1
    else:
        i_prev = i - 1

    if j == 0:
        j_prev = -1
    else:
        j_prev = j - 1

    if i == len(solution) - 1:
        i_next = 0
    else:
        i_next = i + 1

    if j == len(solution) - 1:
        j_next = 0
    else:
        j_next = j + 1

    cost_change = (
        tsp.distance_matrix[solution[i_prev], solution[j]]
        + tsp.distance_matrix[solution[j], solution[i_next]]
        + tsp.distance_matrix[solution[j_prev], solution[i]]
        + tsp.distance_matrix[solution[i], solution[j_next]]
        - tsp.distance_matrix[solution[i_prev], solution[i]]
        - tsp.distance_matrix[solution[i], solution[i_next]]
        - tsp.distance_matrix[solution[j_prev], solution[j]]
        - tsp.distance_matrix[solution[j], solution[j_next]]
    )
    return cost_change


def cost_change_intra_edges(tsp, solution, i, j):
    # i is the index of the first node in the first edge
    # j is the index of the second node in the second edge
    # i and j are not adjacent!

    cost_change = (
        tsp.distance_matrix[solution[i], solution[j]]
        + tsp.distance_matrix[solution[i + 1], solution[j + 1]]
        + -tsp.distance_matrix[solution[i], solution[i + 1]]
        - tsp.distance_matrix[solution[j], solution[j + 1]]
    )
    return cost_change


def intra_route_move(tsp, solution, intra_type, method="greedy"):
    intra_nodes = set(solution)
    neighborhood = [(u, v) for u in intra_nodes for v in intra_nodes if u != v]
    np.random.shuffle(neighborhood)

    min_cost = 0
    node_a = node_b = None

    for test_node_a, test_node_b in neighborhood:
        # we can draw first edge with any node, but second have to start from not node_a and not node_a+1 or node_a-1
        if intra_type != "nodes" and abs(test_node_a - test_node_b) == 1:
            continue

        if intra_type == "nodes":
            cost_change = cost_change_intra_nodes(
                tsp,
                solution,
                np.where(solution == test_node_a),
                np.where(solution == test_node_b),
            )
        else:
            cost_change = cost_change_intra_edges(
                tsp, solution, test_node_a, test_node_b
            )

        if cost_change < min_cost:
            min_cost = cost_change
            node_a = test_node_a
            node_b = test_node_b

            if method == "greedy":
                break

    if node_a is not None:
        if intra_type == "nodes":
            index_a = np.where(solution == node_a)[0][0]
            index_b = np.where(solution == node_b)[0][0]

            # Ensure indices are within bounds
            new_index_1 = index_a + 1 if index_a + 1 < len(solution) else 0
            new_index_2 = index_b + 1 if index_b + 1 < len(solution) else 0

            buf = solution[new_index_1].item()
            solution[new_index_1] = solution[new_index_2].item()
            solution[new_index_2] = buf
        else:
            # node_a stays at the same position
            # node_a+1 become node_b
            # node_b become node_a+1
            # node b+1 stays at the same position
            index_a = np.where(solution == test_node_a)[0][0]

            new_index_1 = index_a + 1 if index_a + 1 < len(solution) else 0
            new_index_2 = test_node_b

            buf = solution[new_index_1].item()
            solution[new_index_1] = solution[new_index_2].item()
            solution[new_index_2] = buf

    return solution, min_cost


def inter_route_move(tsp, solution, method="greedy"):
    possible_unselect = set(solution)
    possible_select = set(range(tsp.size)) - possible_unselect
    neighborhood = list(itertools.product(possible_unselect, possible_select))
    np.random.shuffle(neighborhood)

    min_cost = 0
    to_unselect = to_select = None

    for test_unselect, test_select in neighborhood:
        cost_change = cost_change_inter(
            tsp, solution, np.where(solution == test_unselect), test_select
        )

        if cost_change < min_cost:
            min_cost = cost_change
            to_unselect = test_unselect
            to_select = test_select

            if method == "greedy":
                break

    if to_unselect is not None:
        solution[np.where(solution == to_unselect)] = to_select

    return solution, min_cost

## Greedy

1. Choose arbitrarily starting node and generate random/weighted(best heuristic so far) solution
2. Check each move if solution with it it's better
3. Finish, wehn you check all of them and none of them give us better solution.

Randomly selecting solution

1. Randomly choose if inter/intra route move
2. Randomly look for next solution, remembering already visited solutions


In [41]:
def greedy(
    tsp: TspInstance,
    start_node: int,
    initial_solution_getter: callable,
    intra_type: str,
):
    solution = initial_solution_getter(tsp, start_node)

    previous = set()

    while True:
        previous.add(str(solution))
        if np.random.rand() < 0.5:
            solution, cost_change = inter_route_move(tsp, solution)

            if cost_change == 0:
                solution, cost_change = intra_route_move(tsp, solution, intra_type)
        else:
            solution, cost_change = intra_route_move(tsp, solution, intra_type)

            if cost_change == 0:
                solution, cost_change = inter_route_move(tsp, solution)

        if cost_change == 0:
            break

        if cost_change > 0:
            print("positive")
            break

        if str(solution) in previous:
            print("Already seen", solution, " in ", previous)
            break

    return solution

## Steepest

TODO

In [4]:
def steepest(
    tsp: TspInstance,
    start_node: int,
    initial_solution_getter: callable,
    intra_type: str,
):
    solution = initial_solution_getter(tsp, start_node)

    counter = 0
    while True:
        counter += 1

        best_inter_solution, best_inter_cost_change = inter_route_move(
            tsp, solution, "steepest"
        )
        best_intra_solution, best_intra_cost_change = intra_route_move(
            tsp, solution, intra_type, "steepest"
        )

        if best_inter_cost_change == 1 and best_intra_cost_change == 1:
            break

        if best_inter_cost_change < best_intra_cost_change:
            solution = best_inter_solution
            if best_inter_cost_change < 0:
                print(
                    "Inside method iteration no. ",
                    counter,
                    " last cost_change ",
                    best_inter_cost_change,
                    end="\r",
                )
        else:
            solution = best_intra_solution
            if best_intra_cost_change < 0:
                print(
                    "Inside method iteration no. ",
                    counter,
                    " last cost_change ",
                    best_intra_cost_change,
                    end="\r",
                )

    return solution

## Experiments

In [42]:
columns = []
experiments = []

for instance in (TspInstance("TSPA.csv"), TspInstance("TSPB.csv")):
    for method in (greedy, steepest):
        for intra_type in ("nodes", "edges"):
            for initial_solution_getter in (random_solution,):
                column = f"{instance.file_path[3]} {method.__name__} {intra_type} {initial_solution_getter.__name__.split("_")[0]}".upper()
                print(column)
                columns.append(column)
                experiments.append(
                    instance.run_experiments(
                        method, initial_solution_getter, intra_type
                    )
                )

A GREEDY NODES RANDOM
0001/20 Already seen [12  2 16 17  3  9 19 15  8  1]  in  {'[ 1  0  3  8 16 15 12  6  2 14]', '[ 7  8 14 12  2 19 15 11  0  3]', '[ 1  0  3  9 16 15  8 19  2 14]', '[12  2 16 17  3 14 19 15  8  1]', '[ 7 15  8 14  2  1  5 11 10  3]', '[ 1  0  3 16 17 13  8 19  2 14]', '[ 1 15 14  2  7 10  8 12  6  3]', '[ 1  0 16  3 17 13  8  9  2 14]', '[12  2 16 17  3  1 19 15  8 14]', '[ 7  8 15 14  2  1  5 11 10  3]', '[ 1 15  9 19 12  7 14  6  2  3]', '[ 1  0 14  8 16 15 12  6  2  3]', '[ 1 15 19  2 12  7  8  6  9  3]', '[ 2 15  7 14  1  8  0 11  6  3]', '[ 1  0  3  8 16 15  9 19  2 14]', '[ 1  2 16  3 17 13 15  9  8 14]', '[ 7  8 14 12  2 19 15  0 11  3]', '[ 7 15  8  2 14  1  4 11 10  3]', '[ 2 15  8 14  7  1  0 11 10  3]', '[ 1  0 14  9 16 15 12  6  2  3]', '[ 1  0  3  9 16 13  8 19  2 14]', '[ 1  0 19  2 16 15 12  6  9  3]', '[ 1 15  8 19 12 10 14  2  6  3]', '[ 2 15  8 14  7  1  0 11  6  3]', '[ 2 15  1 14  7  8  0 11  6  3]', '[ 1 15  8 12  7 10 14  2  6  3]', '[ 1 15 1

IndexError: index 13 is out of bounds for axis 0 with size 10

In [7]:
pd.DataFrame(
    np.array(tuple(map(lambda x: x[:-1], experiments))).T,
    columns=columns,
    index=(
        "min_cost",
        "max_cost",
        "average_cost",
        "min_time",
        "max_time",
        "average_time",
    ),
)

,Greedy
min,8518.000000
max,11046.000000
avg,9996.000000
min_time,0.028960
max_time,0.230561
avg_time,0.085367
